In [1]:
import random
import gradio as gr
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

c:\Users\Alex School\.conda\envs\ml_3_10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bot = load_model('bots/blackjack_bot_5000epochs.h5')

In [3]:
# Game logic functions
def create_deck():
    deck = []
    for suit in ['hearts', 'diamonds', 'clubs', 'spades']:
        for rank in range(2, 11):
            deck.append((str(rank), suit))
        for face in ['jack', 'queen', 'king', 'ace']:
            deck.append((face, suit))
    random.shuffle(deck)
    return deck

def calculate_score(hand):
    value_map = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'jack': 10, 'queen': 10, 'king': 10, 'ace': 11}
    score = sum(value_map[card[0]] for card in hand)
    aces = sum(1 for card in hand if card[0] == 'ace')
    while score > 21 and aces:
        score -= 10
        aces -= 1
    return score

def has_usable_ace(hand):
    """Check if the hand contains a usable ace (i.e., can count as 11 without busting)."""
    score = calculate_score(hand)
    # A usable ace is one where the ace is counted as 11 and the total score doesn't exceed 21
    return any(card[0] == 'ace' for card in hand) and score <= 21

def get_state(player_hand, dealer_hand):
    player_hand_sum = calculate_score(player_hand)
    dealer_hand_sum = calculate_score(dealer_hand[1:]) # Assumes the dealer's first card will be the face-down one, so this excludes from the sum that the bot will see
    player_usable_ace = int(has_usable_ace(player_hand))
    return np.array([player_hand_sum, dealer_hand_sum, player_usable_ace])

def bot_decision(bot, state):
    decision = bot.predict(state[np.newaxis], verbose=False)
    return "Hit" if np.argmax(decision) == 1 else "Stand"

# Initialize global state
deck = []
player_hand = []
dealer_hand = []
machine_hand = []
def reset_game():
    global deck, player_hand, dealer_hand, machine_hand
    deck = create_deck()
    player_hand = [deck.pop(), deck.pop()]
    dealer_hand = [deck.pop(), deck.pop()]
    machine_hand = [deck.pop(), deck.pop()]
    player_images = [resize_image(get_card_image(card)) for card in player_hand]
    dealer_images = [resize_image(get_card_image(dealer_hand[0]))]
    machine_images = [resize_image(get_card_image(card)) for card in machine_hand]
    result = "Game in progress..."
    player_score = calculate_score(player_hand)
    dealer_score = calculate_score(dealer_hand[:1])
    machine_score = calculate_score(machine_hand)
    recommendation = get_recommendation(player_score, dealer_hand[0][0])
    return player_images, dealer_images, machine_images, result, player_score, dealer_score, machine_score, recommendation

# Function to get the image paths for the cards
def get_card_image(card):
    rank, suit = card
    return f"cards/{suit}_{rank}.png"

# Resize card image to be smaller
def resize_image(img_path, size=(400, 600)):
    img = Image.open(img_path)
    img = img.resize(size)
    return img

def play_blackjack(player_action):
    global player_hand, dealer_hand, machine_hand, deck
    if player_action == "Hit":
        player_hand.append(deck.pop())
    player_score = calculate_score(player_hand)
    dealer_score = calculate_score(dealer_hand[:1])
    machine_score = calculate_score(machine_hand)
    
    # Machine player logic
    # Get the state data for the bot to use
    machine_state = get_state(machine_hand, dealer_hand)

    # Bot decides its next move based on the state
    if bot_decision(bot, machine_state) == 'Hit':
        machine_hand.append(deck.pop())
        machine_score = calculate_score(machine_hand)
    # while machine_score < 17:
    #     machine_hand.append(deck.pop())
    #     machine_score = calculate_score(machine_hand)
    result = "Game in progress..."
    recommendation = get_recommendation(player_score, dealer_hand[0][0])
    if player_action == "Stand" or (len(player_hand) > 2 and player_score > 21):
        while calculate_score(dealer_hand) < 17:
            dealer_hand.append(deck.pop())
        dealer_score = calculate_score(dealer_hand)
        player_bust = player_score > 21
        dealer_bust = dealer_score > 21
        machine_bust = machine_score > 21
        # Determine the outcome after the player stands or busts
        if player_bust and dealer_bust and machine_bust:
            result = "All players bust! It's a tie."
        elif player_bust and dealer_bust:
            result = "Player and Dealer bust. Machine wins!"
        elif player_bust and machine_bust:
            result = "Player and Machine bust. Dealer wins!"
        elif dealer_bust and machine_bust:
            result = "Dealer and Machine bust. Player wins!"
        elif player_bust:
            if machine_bust:
                result = "Player busts. Dealer wins."
            else:
                result = "Player busts. Machine wins against Dealer."
        elif dealer_bust:
            result = "Dealer busts. Both Player and Machine win!"
        elif machine_bust:
            result = "Machine busts. Player wins against Dealer."
        else:
            player_result = "Player wins" if player_score > dealer_score else "Dealer wins" if dealer_score > player_score else "Player and Dealer push"
            machine_result = "Machine wins" if machine_score > dealer_score else "Dealer wins" if dealer_score > machine_score else "Machine and Dealer push"
            result = f"Player vs Dealer: {player_result}. Machine vs Dealer: {machine_result}"
        recommendation = "Game over!"
    return update_ui(result, player_score, dealer_score, machine_score), recommendation

def update_ui(result="Game in progress...", player_score=0, dealer_score=0, machine_score=0):
    player_images = [resize_image(get_card_image(card)) for card in player_hand]
    if result == "Game in progress...":
        dealer_images = [resize_image(get_card_image(dealer_hand[0]))]
    else:
        dealer_images = [resize_image(get_card_image(card)) for card in dealer_hand]
    machine_images = [resize_image(get_card_image(card)) for card in machine_hand]
    return player_images, dealer_images, machine_images, result, player_score, dealer_score, machine_score

# Recommendation logic
def get_recommendation(player_score, dealer_upcard):
    if player_score <= 11:
        return "Hit"
    elif player_score == 12 and dealer_upcard in ['4', '5', '6']:
        return "Stand"
    elif player_score > 11 and player_score < 17 and dealer_upcard in ['2', '3', '4', '5', '6']:
        return "Stand"
    elif player_score >= 17:
        return "Stand"
    else:
        return "Hit"

# Gradio Interface
def blackjack_interface(player_action):
    game_ui, recommendation = play_blackjack(player_action)
    player_images, dealer_images, machine_images, result, player_score, dealer_score, machine_score = game_ui
    return player_images, dealer_images, machine_images, result, player_score, dealer_score, machine_score, recommendation

def restart_game():
    return reset_game()

In [4]:
# Gradio Interface Setup
with gr.Blocks(css=".card-img {width: 400px; height: 300px;}") as demo:
    # Output for Player's Hand and Dealer's Hand
    player_gallery = gr.Gallery(label="Player's Hand", elem_id="player-hand", elem_classes="card-img")
    dealer_gallery = gr.Gallery(label="Dealer's Hand", elem_id="dealer-hand", elem_classes="card-img")
    machine_gallery = gr.Gallery(label="Machine's Hand", elem_id="machine-hand", elem_classes="card-img")
    result_text = gr.Textbox(label="Game Result", elem_id="game-result")
    # Display running totals for Player and Dealer
    player_score_text = gr.Textbox(label="Player's Total Score", elem_id="player-score")
    dealer_score_text = gr.Textbox(label="Dealer's Total Score", elem_id="dealer-score")
    machine_score_text = gr.Textbox(label="Machine's Total Score", elem_id="machine-score")
    # Recommendation text
    recommendation_text = gr.Textbox(label="Recommended Action", elem_id="recommendation")
    # Player Actions (Hit/Stand)
    player_action = gr.Radio(choices=["Hit", "Stand"], label="Choose your action", elem_id="player-action")
    # Buttons for game actions
    action_button = gr.Button("Submit", elem_id="action-btn")
    restart_button = gr.Button("Restart", elem_id="restart-btn")
    # Game action event listener
    action_button.click(
        fn=blackjack_interface,
        inputs=player_action,
        outputs=[player_gallery, dealer_gallery, machine_gallery, result_text, player_score_text, dealer_score_text, machine_score_text, recommendation_text]
    )
    # Restart button event listener
    restart_button.click(
        fn=restart_game,
        outputs=[player_gallery, dealer_gallery, machine_gallery, result_text, player_score_text, dealer_score_text, machine_score_text, recommendation_text]
    )
    # Start the Gradio App
    reset_game()
    demo.launch(debug=True, share=True)